In [5]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import tensorflow_hub as hub
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


max_words = 15000
tokenizer = Tokenizer(num_words=max_words)
maxlen = 150



In [23]:
module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.load("/Users/shantanu/GitHub/capstone/elmo_3")

In [24]:

tokenizer.fit_on_texts(X_train)

def preprocess_text(text_eval):
    sequences = tokenizer.texts_to_sequences([text_eval])
    data = pad_sequences(sequences, maxlen=maxlen)
    return data


NAE_vals_array = np.empty((0, 1024), float)
PAE_vals_array = np.empty((0, 1024), float)

import multiprocessing
manager = multiprocessing.Manager()

final_list_NAE = manager.list()
final_list_PAE = manager.list()

model = load_model('../LSTM_Model.h5')




In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         1500000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 1,668,802
Trainable params: 1,668,802
Non-trainable params: 0
______________________________________________

In [26]:
pwd

'/Users/shantanu/GitHub/capstone/analysis/basic_model'

In [91]:
model_url = '/Users/shantanu/GitHub/capstone/final_models/LSTM_Model.h5'

print("before read")
print(os.path.exists(model_url))
model = load_model(model_url)
print("before read")





before read
True
before read


In [95]:

import time
import random
import os





def abalation(args  ):
    
    pred_vals_array = np.empty((0,2), float)
    NAE_vals_array = np.empty((3,1024), float)
    # NAE_vals_array = []
    PAE_vals_array = np.empty((3,1024), float)

    from tensorflow.keras.models import load_model as lm1
    index, row = args
    print("running for {}".format(index))


    text = row.comment.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens_without_sw = tokens #[word for word in tokens if not word in stopwords.words()]
    

    ### First Get the overall Classification

#     print(tokens_without_sw)
    
    tokens_string = str(tokens_without_sw)
    
    prep_text = preprocess_text(tokens_string)
    
    print(prep_text.shape)
    
    pred_vlaues = model.predict(prep_text)

    classification = pred_vlaues.argmax()
    
    if classification == 0 and row.recipient_attack == 0:
        mode = 'PAE'  # Positive AutoEncoder
    elif classification == 0 and row.recipient_attack == 1:
        mode = 'NAE'
    elif classification == 1 and row.recipient_attack == 0:
        mode = 'PAE'
    elif classification == 1 and row.recipient_attack == 1:
        mode = 'NAE'

    ### Get the Elmo vector
    
    text = tf.convert_to_tensor([str(tokens_without_sw)])
    out = embed.signatures['default'](text)['elmo']

    pred_vals_array = np.empty((0, 2), float)

    print(tokens_without_sw)
    
    for i in range(len(tokens_without_sw) - 2):
        print(i)
        
        text_eval = tokens_without_sw[i:i + 3]
        
        a = preprocess_text(str(text_eval))
        
        print(a)
    
        pred_vlaues = model.predict(a)
        
        print(pred_vlaues)
        
        pred_vals_array = np.vstack((pred_vals_array, pred_vlaues))
        
        
    
    if mode == "NAE":

        # Only Negative Vals, and find the index of the highest contributing phrase

        phrase_start_index = pred_vals_array[:, 1].argmax()

        ### Get the ELMO Encoding of the entire vector:

        elmo_vecs_NAE = np.array(out[0][phrase_start_index:phrase_start_index + 3])

        print(elmo_vecs_NAE)
        
        NAE_vals_array = np.vstack((NAE_vals_array, elmo_vecs_NAE))
        
#         final_list_NAE.append(elmo_vecs_NAE)

        print("NAE")
        print(len(NAE_vals_array))

    elif mode == "PAE":

        # Only Positive Vals, and find the index of the highest contributing phrase

        phrase_start_index = pred_vals_array[:, 0].argmax()
        
        elmo_vecs_PAE = out[0][phrase_start_index:phrase_start_index + 3]

        PAE_vals_array = np.vstack((PAE_vals_array, elmo_vecs_PAE))
        
#         final_list_PAE.append(elmo_vecs_PAE)

        print("PAE")
        print(len(PAE_vals_array))
        
    





In [54]:
root_dir = r'/Users/shantanu/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Wikipedia Data'

result_path = annot_file_path = os.path.join(root_dir , 'comments_with_grouped_annoptations.tsv')

merged_comments = pd.read_table(result_path)
merged_comments['recipient_attack'] = merged_comments['recipient_attack'].apply(lambda x : 1 if x> 1 else 0 )
X_train = merged_comments['comment']

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)


data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = merged_comments['recipient_attack']

one_hot_train_labels = to_categorical(y)

labels = np.asarray(one_hot_train_labels)
print('Shape of label tensor:', labels.shape)
labels = labels[indices]


X_train, X_test_validate, y_train, y_test_validate = train_test_split(merged_comments.comment, merged_comments.recipient_attack,
                                                    stratify=labels,
                                                    test_size=0.995,
                                                    random_state=123)

DF_train = X_train.to_frame()
DF_train['recipient_attack'] = y_train

DF_train.reset_index(inplace=True)

from multiprocessing import Pool



Shape of data tensor: (115864, 150)
Shape of label tensor: (115864, 2)


In [30]:

def run_multiproc ():
    with Pool(1) as p:
        p.map(abalation,DF_train.iloc[1:2,:].iterrows() )


In [96]:
def run_sequential ():
    for rec in DF_train.iterrows():
        abalation(rec)


In [97]:
run_sequential()

running for 0
(1, 150)
['NEWLINETOKENFluffernutter', 'I', 'dont', 'think', 'this', 'needs', 'mediation', 'thanks', 'Scotty', 'is', 'a', 'big', 'boy', 'and', 'so', 'am', 'I', 'We', 'disagree', 'over', 'something', 'and', 'we', 'could', 'have', 'handled', 'that', 'very', 'well', 'I', 'suppose', 'until', 'someone', 'came', 'in', 'between', 'and', 'started', 'throwing', 'blocks', 'about', 'Scotty', 'Im', 'sorry', 'for', 'what', 'I', 'said', 'you', 'hurt', 'my', 'feelings', 'and', 'I', 'let', 'it', 'be', 'known', 'Im', 'not', 'going', 'to', 'come', 'back', 'here', 'since', 'I', 'dont', 'want', 'to', 'bother', 'you', 'you', 'can', 'always', 'drop', 'me', 'a', 'line', 'on', 'my', 'talk', 'page', 'and', 'youre', 'welcome', 'to', 'leave', 'a', 'TB', 'template', 'as', 'well', 'pace', 'DNTR', 'Take', 'it', 'easy']
0
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

KeyboardInterrupt: 

In [94]:
a

NameError: name 'a' is not defined

In [31]:
DF_train.reset_index(inplace=True,drop=True)

In [32]:
DF_train.iloc[44]

index                                                           74266
comment             NEWLINE_TOKENNEWLINE_TOKEN== Eridge railway st...
recipient_attack                                                    0
Name: 44, dtype: object

In [12]:
PAE_vals_array

array([], shape=(0, 1024), dtype=float64)

In [83]:
prep_text = preprocess_text(DF_train.iloc[35].comment)



In [84]:
DF_train.iloc[35].comment

"`NEWLINE_TOKENNEWLINE_TOKEN== Bye Bye Jerk! ==NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENI guess this means you will be crying about the Nazis anymore.  What is your problem with Nazis anyway?  Did you date a Nazi woman and she broke your heart??  What a shame!!!NEWLINE_TOKENNEWLINE_TOKENYour life isn't worth living anymore so pleasee KILL YOURSELF!NEWLINE_TOKENNEWLINE_TOKENYou know I'm right.  You've felt this way for a long time.  You've probably been thinking about killing yourself for quite some time.  I'm here to tell you, it's all right.  Go ahead.  Do it.  It won't hurt. Be brave.  All your troubles will be over.  You won't have the pain anymore.  Everyone else?  Don't worry, they'll get by.  NEWLINE_TOKENNEWLINE_TOKENWhat do you mean It's not your fault things turned out badly?  It certainly IS your fault.  You ought to be ashamed of yourself for what you've done  - and failed to do.  Maybe things could have changed if you weren't such  a weakling.  You're defective. 

In [85]:
prep_text

array([[   88,    87,    68,     3,   316,     8,    81,    41,   134,
          106,  1160,    36,    14,    14,   572,  2423,    19,  6431,
           41,    23,  8012,    44,    19,   129,     8,   572,    22,
            1,  2963,  1208,   512,   304,    58,  2051,  4873,    97,
           32,     2,     4,   755,    36,     8,   279,    81,    16,
           23,  2736,   214,  1954,    80,  3469,    14,   585,    11,
           23,  2736,     8,  2376,     3,    19,  4224,     6,   183,
           15,    42,   575,   223,     5,  1371,     3,    36,   376,
          214,   112,    22,   507,    28,     8,  2460,   110,     7,
          181, 11416,   575,    56,    12,   108,    15,     7,   255,
           88,     8,    20,   751,    12,   518,   396,  1856,     2,
            4,  3529,    39,    23,   272,  2575,   155,  2575,   155,
         2575,     5,    59,    61,   397,   138,   214,    12,     8,
           92,     8,    70,    42,    87,   536,    38,    10,   221,
      

In [86]:

pred_vlaues = model.predict(prep_text)



In [87]:
pred_vlaues
    

array([[0.16728354, 0.82756335]], dtype=float32)

In [90]:
text = DF_train.iloc[35].comment.translate(str.maketrans('', '', string.punctuation))
tokens = word_tokenize(text)
tokens_without_sw = tokens #[word for word in tokens if not word in stopwords.words()]

### First Get the overall Classification

print(tokens_without_sw)

tokens_string = str(tokens_without_sw)

print("2")

prep_text = preprocess_text(tokens_string)

print("3")

pred_vlaues = model.predict(prep_text)

print("4")

['NEWLINETOKENNEWLINETOKEN', 'Bye', 'Bye', 'Jerk', 'NEWLINETOKENNEWLINETOKENNEWLINETOKENNEWLINETOKENI', 'guess', 'this', 'means', 'you', 'will', 'be', 'crying', 'about', 'the', 'Nazis', 'anymore', 'What', 'is', 'your', 'problem', 'with', 'Nazis', 'anyway', 'Did', 'you', 'date', 'a', 'Nazi', 'woman', 'and', 'she', 'broke', 'your', 'heart', 'What', 'a', 'shameNEWLINETOKENNEWLINETOKENYour', 'life', 'isnt', 'worth', 'living', 'anymore', 'so', 'pleasee', 'KILL', 'YOURSELFNEWLINETOKENNEWLINETOKENYou', 'know', 'Im', 'right', 'Youve', 'felt', 'this', 'way', 'for', 'a', 'long', 'time', 'Youve', 'probably', 'been', 'thinking', 'about', 'killing', 'yourself', 'for', 'quite', 'some', 'time', 'Im', 'here', 'to', 'tell', 'you', 'its', 'all', 'right', 'Go', 'ahead', 'Do', 'it', 'It', 'wont', 'hurt', 'Be', 'brave', 'All', 'your', 'troubles', 'will', 'be', 'over', 'You', 'wont', 'have', 'the', 'pain', 'anymore', 'Everyone', 'else', 'Dont', 'worry', 'theyll', 'get', 'by', 'NEWLINETOKENNEWLINETOKENWhat',

In [89]:
pred_vlaues

array([[0.75921273, 0.21199068]], dtype=float32)

In [91]:
prep_text.shape

(1, 150)